In [31]:
import pandas as pd
import random
from scipy.optimize import minimize
from datetime import datetime
import numpy as np

In [23]:
# Function to generate dummy data
def generate_data(end_date=datetime.now(), num_steps=100, collection_name = 'BAYC'):
    """
    This function generates a dummy dataset with the specified columns.
    """
    df = pd.DataFrame({
        'time': pd.date_range(end=end_date, periods=num_steps, freq='T').round('T'),  # num_steps minutes ending at end_date
        'price_nftx': [random.randint(100, 1000) for _ in range(num_steps)],
        'price_market_place_x': [random.randint(100, 1000) for _ in range(num_steps)],
        'price_market_place_y': [random.randint(100, 1000) for _ in range(num_steps)],
        'price_market_place_z': [random.randint(100, 1000) for _ in range(num_steps)],
        'collection_name': [collection_name for _ in range(num_steps)],  # Only one collection named BAYC
        'fee_nftx': [random.random() * 10 for _ in range(num_steps)],
        #'blur_price': [random.randint(100, 1000) for _ in range(num_steps)],
        'fee_market_place_x': [random.random() * 10 for _ in range(num_steps)],
        'fee_market_place_y': [random.random() * 10 for _ in range(num_steps)],
        'fee_market_place_z': [random.random() * 10 for _ in range(num_steps)],
    })
    return df

In [24]:
df = generate_data()
df.head()

,time,price_nftx,price_market_place_x,price_market_place_y,price_market_place_z,collection_name,fee_nftx,fee_market_place_x,fee_market_place_y,fee_market_place_z
0,2023-09-12 15:32:00,594,212,753,453,BAYC,6.565650,9.474280,8.422061,5.582765
1,2023-09-12 15:33:00,428,132,326,303,BAYC,6.213809,3.917248,8.083517,3.681309
2,2023-09-12 15:34:00,850,132,774,323,BAYC,8.503785,3.390091,2.132208,2.867715
3,2023-09-12 15:35:00,128,271,639,972,BAYC,0.586741,1.225890,5.844415,8.607843
4,2023-09-12 15:36:00,230,393,216,338,BAYC,7.956737,1.315881,2.101950,3.266617


In [25]:
# Function to compute lowest gross price and minimum net price
def compute_prices(df):
    """
    This function computes the lowest gross price and minimum net price.
    """
    if 'Floor_price' in df.columns:
        df = df.rename(columns={'Floor_price': 'blur_price'})
    # If blur_price is not given, set it to the minimum of price_nftx
    if 'blur_price' not in df.columns or df['blur_price'].isnull().all():
        df['blur_price'] = df['price_nftx'].min()

    # Compute the lowest gross price from the different market places
    df['lowest_gross_price'] = df[['price_market_place_x', 'price_market_place_y', 'price_market_place_z']].min(axis=1)

    # Compute net prices
    df['net_price_x'] = df['price_market_place_x'] - df['fee_market_place_x']
    df['net_price_y'] = df['price_market_place_y'] - df['fee_market_place_y']
    df['net_price_z'] = df['price_market_place_z'] - df['fee_market_place_z']

    # Compute the minimum net price from the different market places
    df['minimum_net_price'] = df[['net_price_x', 'net_price_y', 'net_price_z']].min(axis=1)

    return df

In [26]:
df = compute_prices(df)
df.head()

,time,price_nftx,price_market_place_x,price_market_place_y,price_market_place_z,collection_name,fee_nftx,fee_market_place_x,fee_market_place_y,fee_market_place_z,blur_price,lowest_gross_price,net_price_x,net_price_y,net_price_z,minimum_net_price
0,2023-09-12 15:32:00,594,212,753,453,BAYC,6.565650,9.474280,8.422061,5.582765,102,212,202.525720,744.577939,447.417235,202.525720
1,2023-09-12 15:33:00,428,132,326,303,BAYC,6.213809,3.917248,8.083517,3.681309,102,132,128.082752,317.916483,299.318691,128.082752
2,2023-09-12 15:34:00,850,132,774,323,BAYC,8.503785,3.390091,2.132208,2.867715,102,132,128.609909,771.867792,320.132285,128.609909
3,2023-09-12 15:35:00,128,271,639,972,BAYC,0.586741,1.225890,5.844415,8.607843,102,271,269.774110,633.155585,963.392157,269.774110
4,2023-09-12 15:36:00,230,393,216,338,BAYC,7.956737,1.315881,2.101950,3.266617,102,216,391.684119,213.898050,334.733383,213.898050


In [34]:
# Function to optimize fee
def optimize_fee(df):
    """
    This function optimizes the fee.
    """
    def objective(fee, price_nftx, min_net_price, blur_price):
        net_price = price_nftx - fee
        return -1 * (net_price if net_price <= min_net_price and net_price >= blur_price else 0)

    optimal_fees = []
    for _, row in df.iterrows():
        result = minimize(objective, x0=0, bounds=[(0, row['fee_nftx'])], args=(row['price_nftx'], row['minimum_net_price'], row['blur_price']))
        optimal_fees.append(result.x[0])

    # Add the optimized fee to the dataframe
    df['optimized_fee'] = optimal_fees

    return df

In [35]:
df = optimize_fee(df)
df.head()

,time,price_nftx,price_market_place_x,price_market_place_y,price_market_place_z,collection_name,fee_nftx,fee_market_place_x,fee_market_place_y,fee_market_place_z,blur_price,lowest_gross_price,net_price_x,net_price_y,net_price_z,minimum_net_price,optimized_fee
0,2023-09-12 15:32:00,594,212,753,453,BAYC,6.565650,9.474280,8.422061,5.582765,102,212,202.525720,744.577939,447.417235,202.525720,0.0
1,2023-09-12 15:33:00,428,132,326,303,BAYC,6.213809,3.917248,8.083517,3.681309,102,132,128.082752,317.916483,299.318691,128.082752,0.0
2,2023-09-12 15:34:00,850,132,774,323,BAYC,8.503785,3.390091,2.132208,2.867715,102,132,128.609909,771.867792,320.132285,128.609909,0.0
3,2023-09-12 15:35:00,128,271,639,972,BAYC,0.586741,1.225890,5.844415,8.607843,102,271,269.774110,633.155585,963.392157,269.774110,0.0
4,2023-09-12 15:36:00,230,393,216,338,BAYC,7.956737,1.315881,2.101950,3.266617,102,216,391.684119,213.898050,334.733383,213.898050,0.0


In [36]:
# Function to compare fees
def compare_fees(df):
    """
    This function compares the total fee with the total fee of nftx.
    """
    total_fee = df['optimized_fee'].sum()
    total_fee_nftx = df['fee_nftx'].sum()

    print(f'Total fee: {total_fee}')
    print(f'Total fee nftx: {total_fee_nftx}')

In [38]:
compare_fees(df)

Total fee: 0.0
Total fee nftx: 492.43694274929436
